In [4]:
!pip install tiktoken

In [ ]:
from docx import Document

document = Document('./tax.docx')

full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n'

In [11]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o-mini")
    total_encoding = encoder.encode(full_text)
    
    total_token_count = len(total_encoding)
    
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
         
    return text_list

chunk_list = split_text(full_text, 1500)

In [13]:
import chromadb

chroma_client = chromadb.Client()

collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [7]:
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_embedding = OpenAIEmbeddingFunction(model_name='text-embedding-3-large')

In [12]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)


id_list = [f'{i}' for i in range(len(chunk_list))]
tax_collection.add(documents=chunk_list, ids=id_list)

NameError: name 'chroma_client' is not defined

In [3]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/home/kj/miniconda3/envs/llm/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever = database.as_retriever(),
    chain_type_kwargs = {'prompt': prompt}
)

In [12]:
query = '연봉 5천만원 직장인의 소득세는 얼마인가요?'
ai_message = qa_chain({"query" : query})
ai_message

{'query': '연봉 5천만원 직장인의 소득세는 얼마인가요?',
 'result': '제공된 정보는 소득의 구분, 과세소득의 범위, 특별소득공제 등에 대한 일반적인 세법 규정을 담고 있습니다. 연봉 5천만원 직장인의 구체적인 소득세액을 계산하기 위한 세율, 공제 항목별 금액, 계산 공식 등은 포함되어 있지 않습니다. 따라서 해당 정보만으로는 소득세를 알 수 없습니다.'}